In [0]:
%run "/Workspace/Users/ruchika.b.mhetre@v4c.ai/vstone_proj/00_Setup/project_config"

In [0]:
# Cell 2: Imports and Data Loading
from pyspark.sql.functions import col, count, when

# Load your data
df_main = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{volume_path}/1_main.csv")

display(df_main.limit(5))

In [0]:
# Load primary datasets using variables from project_config
df_main = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{volume_path}/1_main.csv")

# Note: catalogs.csv uses ';' as a delimiter based on your previous output
df_catalogs = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ";") \
    .option("inferSchema", "true") \
    .load(f"{volume_path}/catalogs.csv")

df_geo = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{volume_path}/final_geographic.csv")

In [0]:
display(df_main)

In [0]:
# 1. Print Schema to verify types
print("--- Main Transactions Schema ---")
df_main.printSchema()

# 2. Programmatic Null Check
null_counts = df_main.select([count(when(col(c).isNull(), c)).alias(c) for c in df_main.columns])
print("--- Null Values Count ---")
display(null_counts)

In [0]:
# Check for duplicate IDs which would break your Silver layer joins
duplicate_count = df_main.count() - df_main.select("id").distinct().count()
print(f"Number of duplicate IDs: {duplicate_count}")

if duplicate_count > 0:
    print("WARNING: Duplicate IDs found. These must be handled in the Silver layer.")